# Optimisation zones

Create a collection of boolean "zones" (intervals) to cover the domain of a "continuous" optimisation variable.
Sum the zones to reconstruct the bounding interval for the continuous value.

In [1]:
from ortools.sat.python import cp_model
import random

In [31]:
model = cp_model.CpModel()
n = 10
resolution = 10000

Proxy continuous variable and some arbitrary constraints

In [32]:
floats = [model.NewIntVar(0, resolution, f"f_{i}") for i in range(n)]
model.Add(sum(floats) == 40000)
for i in random.sample(range(n), 3):
    model.Add(floats[i] < 7000)
for i in random.sample(range(n), 3):
    model.Add(floats[i] > 3000)

Define the zones, in this case 10 zones

In [33]:
w = 10
zones = [[model.NewBoolVar(f"z_{i}_{z}") for z in range(w)] for i in range(n)] 
for i in range(n):
    for z in range(w):
        model.Add(floats[i] > z * resolution // w).OnlyEnforceIf(zones[i][w - z - 1])
        model.Add(floats[i] <= z * resolution // w).OnlyEnforceIf(zones[i][w - z - 1].Not())

Demonstrate the zones match continuous values and bounding interval can be reconstructed

In [38]:
solver = cp_model.CpSolver()
status = solver.solve(model)
print("solution", status)
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    float_solution = [solver.value(f) for f in floats]
    zones_solution = [[solver.value(z) for z in zs] for zs in zones]
    print("continous", float_solution)
    print("zones", zones_solution)
    print("reconstructed", [(max(0, resolution * (sum(zs) - 1)), resolution * sum(zs)) for zs in zones_solution])

solution 4
continous [10000, 10000, 6999, 6999, 0, 3001, 3001, 0, 0, 0]
zones [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
reconstructed [(90000, 100000), (90000, 100000), (60000, 70000), (60000, 70000), (0, 0), (30000, 40000), (30000, 40000), (0, 0), (0, 0), (0, 0)]
